#### output weights n layer,  validation set, graph plot, test set, lstm cell stack, preloaded full data before training, fetch data from another notebook, added exection time of 1 epoch and remaining time, training with full data set at once.

In [ ]:
from sklearn import metrics
import tensorflow as tf
#import csv
#from threading import Thread
import time
from matplotlib import pyplot as plt
from  read_proces_data import *

In [ ]:
# hidden units for lstm
hidden_units = 32
# Lstm layers
num_layers = 2
# chunks of input given to lstm at a time for training
batch_size = 5000
# max seq length in training and test data
max_seq = 3260
# input dimension
feature_vec_len = 57

In [ ]:
# find max no of times to roll back
#max_seq_len = find_max_seq_len()
#print max_seq_len
max_seq_len = 3259

In [ ]:
# use ipython feature to read data from another notebook
% store -r train_batch
% store -r seq_batch
% store -r target_batch

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
# create lstm cell in tensorflow
def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(num_units =\
        hidden_units,state_is_tuple=True)
stacked_cell = tf.contrib.rnn.MultiRNNCell(\
        [lstm_cell() for _ in range(num_layers)], \
        state_is_tuple=True)

In [ ]:
x = tf.placeholder(tf.float32, shape = [batch_size,max_seq,feature_vec_len])
s_len = tf.placeholder(tf.int32,shape=[batch_size])
# run lstm over different sequence length
output_, state = tf.nn.dynamic_rnn(\
        stacked_cell,\
        x,\
#        initial_state = i_state,\
        sequence_length=s_len,\
        dtype=tf.float32)
# take all batch, last output of each batch and full output vector
def take_subarray(array , index):
    return array[range(0,batch_size),index-1]
output = tf.py_func(take_subarray,[output_,s_len],tf.float32)
#print output
# define final output value
target_value = tf.placeholder(tf.float32, shape=[batch_size])
# define weights and bias from output of lstm cell to network final output
output_size = stacked_cell.output_size
std = 1/np.sqrt(feature_vec_len,dtype=np.float32)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=std,dtype=tf.float32)
    return tf.Variable(initial,dtype=tf.float32)

def bias_variable(shape):
    initial = tf.constant(std, shape=shape,dtype=tf.float32)
    return tf.Variable(initial,dtype=tf.float32)

Wo1 = weight_variable([output_size, 2 * output_size])
Wo2 = weight_variable([2*output_size, 2 * output_size])
Wo3 = weight_variable([2*output_size, 2 * output_size])
Wo4 = weight_variable([2*output_size, 1])

b1 = bias_variable([2 * output_size])
b2 = bias_variable([2 * output_size])
b3 = bias_variable([2 * output_size])
b4 = bias_variable([1])

# initialize variables
# calculate final output

s1  = tf.matmul(output, Wo1) + b1
o1 = tf.nn.relu(s1)

s2  = tf.matmul(o1, Wo2) + b2
o2 = tf.nn.relu(s2)

s3  = tf.matmul(o2, Wo3) + b3
o3 = tf.nn.relu(s3)

logits  = tf.matmul(o3, Wo4) + b4
observed = tf.sigmoid(logits)
#observed = tf.nn.relu(logits)

# calculate cost as calculated in logistic classification.
logistic_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(\
        labels = tf.reshape(target_value,[batch_size,1]), logits = logits))

# optimize the cost
train_step = tf.train.AdamOptimizer(1e-4).minimize(logistic_cost)

def threshold_fn(array):
    return array >= 0.50
obser = tf.py_func(threshold_fn,[observed], tf.bool)
observe = tf.cast(obser,tf.float32)

sess.run(tf.global_variables_initializer())

In [ ]:
def calc_metrics(t,y):
    
    f1score = metrics.f1_score(t,y)
    
    fprr, tprr, thresholds = metrics.roc_curve(t, y, pos_label=1)
    aucc = metrics.auc(fprr,tprr)
    
    sk_acc = metrics.accuracy_score(t,y)
    
    return (f1score, aucc, sk_acc)

In [ ]:
def evaluate_(data, is_train):
    # evaluate on all last 5000 values but
    # take only last validation set values
    y = sess.run(observe, feed_dict = {\
        x:data[0], s_len:data[1]})
    if is_train:
        c = sess.run(logistic_cost, feed_dict = {target_value:data[2],\
            x:data[0], s_len:data[1]})
        f1s,auc,acc = calc_metrics(data[2],y)
        train_para.append([f1s, auc, acc, c])
    else :
        f1s,auc,acc = calc_metrics(data[2][-440:],y[-440:])
        vali_para.append([f1s, auc, acc])

In [ ]:
def train_(data):
    # train on training data-set
    sess.run(train_step, feed_dict = {target_value:data[2],\
            x:data[0], s_len:data[1]})

In [ ]:
def plot_parameters(training_p, validation_p):
    nu_para = training_p.shape[1]
    para_name = ['f1score','auc','accuracy']
    for j in range(nu_para):
        plt.subplot(nu_para,1,j+1)
        plt.plot(training_p[:,j],'r')
        plt.plot(validation_p[:,j],'b')
        plt.ylabel(para_name[j])
        plt.grid()
    plt.show()

In [ ]:
# number of times to iterate over whole training dataset    
iter_over_files =  1000
# train_para and vali_para are storing f1score,auc,accuracy for 
# train and validation set.
train_para = []
vali_para = []

now_time = time.time()
for itera in range(0, iter_over_files):
    print "iteration %d"%itera
        
    train_data = [train_batch[:5000], seq_batch[:5000], target_batch[:5000]]
    train_(train_data)
    
    if 50 == itera:
        validation_data = [train_batch[-5000:], seq_batch[-5000:], target_batch[-5000:]]
        evaluate_(validation_data, 0)
        evaluate_(train_data, 1)
        
        time_taken = (time.time()-now_time)/60.0
        print "time(in minutes) for 50 iter %f"%(time_taken)
        print "remaining time(in minutes) %f"%((time_taken/50.0)*(iter_over_files-1-itera))
        now_time = time.time()

t_para = np.asarray(train_para)
v_para = np.asarray(vali_para)

plot_parameters(t_para, v_para)

In [ ]:
# predict values for test data set
print ('opening file for write')
label = ["IdFeedBack,Prediction"]
with open('output.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ')
    spamwriter.writerow(label)

    for file_ in test_file_dir:
        # get features from current indexed file
        features = get_test_data(file_)        
        # training features is np array with max size and padding
        test_features, seq_len = prepare_data(features, max_seq_len)
        # scale features to have 0 mean and 1 variance
        scaled_features = feature_scaling(test_features, batch_size,\
                seq_len)
        # find independent components from given features
        inde_features = independent_components(scaled_features, batch_size,\
                seq_len)
        # evaluate over whole file
        y = np.zeros(0)
        if "Sess05" in file_:
            indices = ["%03d" %(i+1) for i in range(100)]
    #        print file_
            for index in range(0,100,batch_size):
                yfile = observe.eval(feed_dict = {\
                    x:inde_features[index:index+batch_size], \
                    s_len:seq_len[index:index+batch_size]})
                y = np.append(y,yfile)
        else:
            indices = ["%03d" %(i+1) for i in range(60)]
            for index in range(0,60,batch_size):
                yfile = observe.eval(feed_dict = {\
                    x:inde_features[index:index+batch_size], \
                    s_len:seq_len[index:index+batch_size]})
                y = np.append(y,yfile)

        for i in xrange(0,y.shape[0]):
            values = [file_[5:-4]+'_FB'+indices[i]+','+str(int(y[i]))]
            spamwriter.writerow(values)
print ('file write complete')